## Restaurant Practice 2 ##

Dataset: restaurant_transactions.xlsx

In [1]:
## Step 1: Import Dependencies

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
## Step 2: Load Data

df = pd.read_excel("restaurant_transactions.xlsx")

print(f"Shape of data: {df.shape}")
print(df.head())

Shape of data: (3078, 23)
   TransactionID CustomerID   Restaurant  Location       Date      Time  \
0          10001   CUST0093     TacoTime   Chicago 2024-11-02  11:18:00   
1          10002   CUST0089      PizzaCo  New York 2024-11-02  13:27:00   
2          10003   CUST0092     TacoTime   Houston 2024-11-02  18:34:00   
3          10004   CUST0014  BurgerPlace   Houston 2024-11-02  12:15:00   
4          10005   CUST0089      PizzaCo  New York 2024-11-02  12:10:00   

    OrderType PaymentMethod  \
0     Dine-in     Gift Card   
1    Delivery          Cash   
2     Dine-in    Mobile App   
3     Takeout   Credit Card   
4  Drive-thru   Credit Card   

                                        ItemsOrdered  Subtotal  ...  \
0                         Beans, Quesadilla, Burrito     19.47  ...   
1  Veggie Pizza, Chicken Wings, Meat Lovers Pizza...     51.96  ...   
2                                 Beef Taco, Burrito     12.98  ...   
3                                       Cheeseburger

In [3]:
print("\nData Types: ")
print(df.dtypes)


Data Types: 
TransactionID                     int64
CustomerID                       object
Restaurant                       object
Location                         object
Date                     datetime64[ns]
Time                             object
OrderType                        object
PaymentMethod                    object
ItemsOrdered                     object
Subtotal                        float64
Discount                        float64
Tax                             float64
Tip                             float64
DeliveryFee                     float64
Total                           float64
ProcessingTimeMin               float64
EmployeeID                       object
Weather                          object
PromotionCode                    object
SatisfactionRating              float64
TransactionType                  object
RefundReason                     object
OriginalTransactionID           float64
dtype: object


In [4]:
print(df.isnull().sum())

TransactionID               0
CustomerID                  0
Restaurant                  0
Location                    0
Date                        0
Time                        0
OrderType                   0
PaymentMethod               0
ItemsOrdered                0
Subtotal                    0
Discount                    0
Tax                         0
Tip                         0
DeliveryFee                 0
Total                       0
ProcessingTimeMin          11
EmployeeID                 10
Weather                     9
PromotionCode            2497
SatisfactionRating       2067
TransactionType             0
RefundReason             2917
OriginalTransactionID    2917
dtype: int64


In [5]:
print("\nUnique Restaurants: ")
print(df['Restaurant'].unique())
print("\nUnique Payment Methods:")
print(df['PaymentMethod'].unique())


Unique Restaurants: 
['TacoTime' 'PizzaCo' 'BurgerPlace' 'SandwichShop' 'BurgerPlace.'
 'Pizza Co' 'Pizza Co.' 'Burger Place' 'Burger_Place' 'PizzaCo.']

Unique Payment Methods:
['Gift Card' 'Cash' 'Mobile App' 'Credit Card' 'Debit Card' 'GIFT CARD'
 'credit card' 'gift card' 'cash' 'MOBILE APP' 'mobile app' 'CASH'
 'debit card' 'DEBIT CARD' 'CREDIT CARD']


# Data Quality Issues Added

This updated code includes several realistic data quality issues that you'll need to clean:

1. **Refund Transactions**:
   * Added refund transactions (about 5% of transactions) with negative amounts
   * Linked to original transactions via OriginalTransactionID
   * Only applied to credit card transactions

2. **Inconsistent Formatting**:
   * Restaurant names have variations (e.g., "PizzaCo", "Pizza Co", "Pizza Co.")
   * Payment methods have inconsistent capitalization

3. **Missing Values**:
   * Some rows have NULL values for Weather, EmployeeID, or ProcessingTimeMin

4. **Duplicate Transactions**:
   * About 1% of transactions are duplicated with new transaction IDs

5. **Calculation Errors**:
   * Some refund transactions have slight discrepancies between components and totals

In [6]:
# Step 3: Clean Data

# 3a: Standardize restaurant names
print(df['Restaurant'].value_counts())

# Create mapping dictionary
restaurant_mapping = {
    "Pizza Co" : "PizzaCo",
    "Pizza Co." : "PizzaCo",
    'PizzaCo.' : 'PizzaCo',
    'Burger Place' : 'BurgerPlace',
    'BurgerPlace.' : 'BurgerPlace',
    'Burger_Place' : 'BurgerPlace'
}

df['Restaurant'] = df['Restaurant'].replace(restaurant_mapping)

print(df['Restaurant'].value_counts())

Restaurant
PizzaCo         821
TacoTime        792
SandwichShop    746
BurgerPlace     688
Burger Place      9
Pizza Co          7
BurgerPlace.      5
Pizza Co.         4
Burger_Place      4
PizzaCo.          2
Name: count, dtype: int64
Restaurant
PizzaCo         834
TacoTime        792
SandwichShop    746
BurgerPlace     706
Name: count, dtype: int64


In [7]:
# 3b: normalize payment methods
print(df['PaymentMethod'].value_counts())

payment_mapping = {
    'gift card' : 'Gift Card',
    'credit card' : 'Credit Card',
    'GIFT CARD' : 'Gift Card',
    'MOBILE APP' : 'Mobile App',
    'CREDIT CARD' : 'Credit Card',
    'debit card' : 'Debit Card',
    'mobile app' : 'Mobile App',
    'CASH' : 'Cash',
    'cash' : 'Cash',
    'DEBIT CARD' : 'Debit Card'
}

df['PaymentMethod'] = df['PaymentMethod'].replace(payment_mapping)
print("\nAfter replacement: ")
print(df['PaymentMethod'].value_counts())



PaymentMethod
Credit Card    711
Debit Card     619
Cash           566
Gift Card      562
Mobile App     529
gift card       15
credit card     14
GIFT CARD       11
MOBILE APP      10
CREDIT CARD      9
debit card       8
mobile app       7
CASH             7
cash             5
DEBIT CARD       5
Name: count, dtype: int64

After replacement: 
PaymentMethod
Credit Card    734
Debit Card     632
Gift Card      588
Cash           578
Mobile App     546
Name: count, dtype: int64


In [ ]:
# 3c: Missing values
missing_cols = df.columns[df.isnull().any()].tolist()
print(f"Columns with missing values: {missing_cols}")

# Strategy for missing values
if 'Weather' in missing_cols:
    # Fill in missing weather with most common weather
    most_common_weather = df['Weather'].mode()[0]
    df['Weather'] = df['Weather'].fillna(most_common_weather)
    print('Filled missing weather with most common weather: ', most_common_weather)

if 'EmployeeID' in missing_cols:
    # Fill in missing employee ID with Unknown
    df['EmployeeID'] = df['EmployeeID'].fillna('Unknown')
    print("Filled missing employee ID with Unknown")

if 'ProcessingTimeMin' in missing_cols:
    # Fill in processing tiime with median based on order type
    for order_type in df['OrderType'].unique():
        median_processing_time = df[df['OrderType'] ==order_type]['ProcessingTimeMin'].median()
        df.loc[(df['OrderType'] == order_type) & 
               (df['ProcessingTimeMin'].isnull()), 'ProcessingTimeMin'] = median_processing_time
        print(f"Filled Missing processing time for {order_type} with: {median_processing_time}")



Columns with missing values: ['ProcessingTimeMin', 'EmployeeID', 'Weather', 'PromotionCode', 'SatisfactionRating', 'RefundReason', 'OriginalTransactionID']
Filled missing weather with most common weather:  Sunny
Filled missing employee ID with Unknown
Filled Missing processing time for Dine-in with: 29.0
Filled Missing processing time for Delivery with: 42.0
Filled Missing processing time for Takeout with: 16.0
Filled Missing processing time for Drive-thru with: 9.0


In [ ]:
# 3d: remove duplicates

# 3e: validate transaction totals (check if total = subtotal - discount + tax + tip + delivery fee)



In [9]:
# 3f: handle refunds
refunds = df[df['Total'] < 0]
print(f"Number of refund transactions: {len(refunds)}")




# Match refunds with original transaction

# remove refunds

Number of refund transactions: 161
